In [1]:
import eli5
from eli5.sklearn import PermutationImportance
import pandas as pd
import numpy as np
import warnings
import datetime
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from sklearn import preprocessing
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, cohen_kappa_score
import lightgbm as lgb
from functools import partial
import json
import copy
import time
import seaborn as sns
import scipy as sp
import xgboost as xgb
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows",1000)
np.set_printoptions(precision=8)
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.


In [2]:
def qwk(a1, a2):
    max_rat = 3
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)
    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))
    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)
    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)
    e = e / a1.shape[0]
    return np.round(1 - o / e, 8)

In [3]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])
        return -qwk(y, X_p)
    
    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')
        
    def predict(self, X, coef):
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])

    def coefficients(self):
        return self.coef_['x']

# install

In [4]:
%%time
train = pd.read_csv('../input/data-science-bowl-2019/train.csv')
train_labels = pd.read_csv('../input/data-science-bowl-2019/train_labels.csv')
test = pd.read_csv('../input/data-science-bowl-2019/test.csv')
#specs = pd.read_csv('../input/data-science-bowl-2019/specs.csv')
sample_submission = pd.read_csv('../input/data-science-bowl-2019/sample_submission.csv')

CPU times: user 1min 8s, sys: 10.4 s, total: 1min 19s
Wall time: 1min 19s


In [5]:
keep_id = train[train.type == "Assessment"][['installation_id']].drop_duplicates()
train = pd.merge(train, keep_id, on="installation_id", how="inner")
train = train[train.installation_id.isin(train_labels.installation_id.unique())]
assess_title = ['Mushroom Sorter (Assessment)', 'Bird Measurer (Assessment)',
       'Cauldron Filler (Assessment)', 'Cart Balancer (Assessment)', 'Chest Sorter (Assessment)']
def remove_index_calc(df):
    additional_remove_index = []
    for i, session in df.groupby('installation_id', sort=False):
        last_row = session.index[-1]
        session = session[session.title.isin(assess_title)]
        first_row = session.index[-1] + 1
        for j in range(first_row, last_row+1):
            additional_remove_index.append(j)                
    return additional_remove_index
additional_remove_index = remove_index_calc(train)
train = train[~train.index.isin(additional_remove_index)]

# Preprocess and Feature engineering

In [6]:
%%time
def encode_title(train, test):
    #train = pd.merge(train, length_df, on="title", how="left")
    #test = pd.merge(test, length_df, on="title", how="left")
    #train.fillna(-999)
    #test.fillna(-999)
    
    train['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), train['title'], train['event_code']))
    test['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), test['title'], test['event_code']))
    list_of_title_eventcode = list(set(train['title_event_code'].unique()).union(set(test['title_event_code'].unique())))

    list_of_user_activities = list(set(train['title'].unique()).union(set(test['title'].unique())))
    list_of_event_code = list(set(train['event_code'].unique()).union(set(test['event_code'].unique())))
    list_of_worlds = list(set(train['world'].unique()).union(set(test['world'].unique())))
    activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))
    activities_labels = dict(zip(np.arange(len(list_of_user_activities)), list_of_user_activities))
    activities_world = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    assess_titles = list(set(train[train['type'] == 'Assessment']['title'].value_counts().index).union(set(test[test['type'] == 'Assessment']['title'].value_counts().index)))

    train['title'] = train['title'].map(activities_map)
    test['title'] = test['title'].map(activities_map)
    train['world'] = train['world'].map(activities_world)
    test['world'] = test['world'].map(activities_world)

    win_code = dict(zip(activities_map.values(), (4100*np.ones(len(activities_map))).astype('int')))
    win_code[activities_map['Bird Measurer (Assessment)']] = 4110
    
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    #train['date'] = train['timestamp'].dt.date
    #test['date'] = test['timestamp'].dt.date
    
    #train["x"] = train["event_data"].apply(lambda x: json.loads(x)["coordinates"]["x"] if "\"coordinates\"" in x else -999)
    #train["y"] = train["event_data"].apply(lambda x: json.loads(x)["coordinates"]["y"] if "\"coordinates\"" in x else -999)
    #test["x"] = test["event_data"].apply(lambda x: json.loads(x)["coordinates"]["x"] if "\"coordinates\"" in x else -999)
    #test["y"] = test["event_data"].apply(lambda x: json.loads(x)["coordinates"]["y"] if "\"coordinates\"" in x else -999)
    
    #train["target_distances"] = train["event_data"].apply(lambda x: sum(json.loads(x)["target_distances"]) if "\"target_distances\"" in x else np.nan)
    #test["target_distances"] = test["event_data"].apply(lambda x: sum(json.loads(x)["target_distances"]) if "\"target_distances\"" in x else np.nan)
    #train["distance"] = train["event_data"].apply(lambda x: json.loads(x)["distance"] if "\"distance\"" in x else np.nan)
    #test["distance"] = test["event_data"].apply(lambda x: json.loads(x)["distance"] if "\"distance\"" in x else np.nan)
    train["misses"] = train["event_data"].apply(lambda x: json.loads(x)["misses"] if "\"misses\"" in x else np.nan)
    test["misses"] = test["event_data"].apply(lambda x: json.loads(x)["misses"] if "\"misses\"" in x else np.nan)
    #train["group"] = train["event_data"].apply(lambda x: json.loads(x)["group"] if "\"group\"" in x else np.nan)
    #test["group"] = test["event_data"].apply(lambda x: json.loads(x)["group"] if "\"group\"" in x else np.nan)
    
    #train_timediff = train['timestamp'].diff(1) / np.timedelta64(1,'s')
    #train_timediff = list(train_timediff[1:])
    #train_timediff.append(0)
    #train["timediff"] = train_timediff
    #test_timediff = test['timestamp'].diff(1) / np.timedelta64(1,'s')
    #test_timediff = list(test_timediff[1:])
    #test_timediff.append(0)
    #test["timediff"] = test_timediff
    
    #train["clip_video"] = train["title"].apply(lambda x: "magma_intro" if x in magma_short_video else(
    #                                                "magma_long" if x in magma_long_video else(
    #                                                "tree_intro" if x in tree_short_video else(
    #                                                "tree_long" if x in tree_long_video else(
    #                                                "crys_intro" if x in crys_short_video else(
    #                                                "crys_long" if x in crys_long_video else "Non_video"))))))
    #test["clip_video"] = test["title"].apply(lambda x: "magma_intro" if x in magma_short_video else(
    #                                                "magma_long" if x in magma_long_video else(
    #                                                "tree_intro" if x in tree_short_video else(
    #                                                "tree_long" if x in tree_long_video else(
    #                                                "crys_intro" if x in crys_short_video else(
    #                                                "crys_long" if x in crys_long_video else "Non_video"))))))
    
    #train["click"] = train["event_data"].apply(lambda x: 1 if "\"coordinates\"" in x else 0)
    #test["click"] = test["event_data"].apply(lambda x: 1 if "\"coordinates\"" in x else 0)
    
    #train["media"] = train["event_data"].apply(lambda x: json.loads(x)["media_type"] if "\"media_type\"" in x else "None")
    #test["media"] = test["event_data"].apply(lambda x: json.loads(x)["media_type"] if "\"media_type\"" in x else "None")
                
    return train, test, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, activities_world, list_of_title_eventcode

train, test, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, activities_world, list_of_title_eventcode = encode_title(train, test)

CPU times: user 32.7 s, sys: 2.63 s, total: 35.3 s
Wall time: 35.3 s


In [7]:
def get_data(user_sample, test_set=False):
    last_activity = 0
    user_activities_count = {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
    time_spent_each_act = {actv: 0 for actv in list_of_user_activities}
    title_eventcode_count = {str(ele): 0 for ele in list_of_title_eventcode}
    
    last_session_time_sec = 0
    all_assessments = []
    accuracy_groups = {"0":0, "1":0, "2":0, "3":0}
    accumulated_accuracy_group = 0
    accumulated_correct_attempts = 0 
    accumulated_uncorrect_attempts = 0 
    accumulated_actions = 0
    counter = 0
    time_first_activity = float(user_sample['timestamp'].values[0])
    durations = []
    miss = 0
    game_true = 0
    game_false = 0
    activity_true = 0
    activity_false = 0
        
    for i, session in user_sample.groupby('game_session', sort=False):      
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        session_title_text = activities_labels[session_title]
        
        # get current session time in seconds
        if session_type != 'Assessment':
            time_spent = int(session['game_time'].iloc[-1] / 1000)
            time_spent_each_act[activities_labels[session_title]] += time_spent   
            
            if session_type == "Game":
                game_true += session['event_data'].str.contains('true').sum()
                game_false += session['event_data'].str.contains('false').sum() 
                
            #if session_type == "Activity":
            #    activity_true += session['event_data'].str.contains('true').sum()
            #    activity_false += session['event_data'].str.contains('false').sum() 

        if (session_type == 'Assessment') & (test_set or len(session)>1): # test set or session in train_label
            all_attempts = session.query(f'event_code == {win_code[session_title]}')
            true_attempts = all_attempts['event_data'].str.contains('true').sum() # true in target assess
            false_attempts = all_attempts['event_data'].str.contains('false').sum() # false in target assessment
            
            # from start of installation_id to the start of target assessment ------------------------
            features = user_activities_count.copy() # appearance of each type without duplicates
            features.update(time_spent_each_act.copy()) # cumulative gameplay time in each title
            features.update(title_eventcode_count.copy()) # apperance of combi of title and event_code
            features['accumulated_correct_attempts'] = accumulated_correct_attempts
            features['accumulated_uncorrect_attempts'] = accumulated_uncorrect_attempts
            accumulated_correct_attempts += true_attempts 
            accumulated_uncorrect_attempts += false_attempts
            features["misses"] = miss
            features['accumulated_actions'] = accumulated_actions
            features["game_true"] = game_true
            features["game_false"] = game_false
            features['game_accuracy'] = game_true / (game_true + game_false) if (game_true + game_false) != 0 else np.nan
            #features["acitivity_true"] = activity_true
            #features["activity_false"] = activity_false
            #features['activity_accuracy'] = activity_true / (activity_true + activity_false) if (activity_true + activity_false) != 0 else np.nan
            
            # from the end of previous assessment to the start of next assessment --------------------------
                        
            # unique type --------------------------------------------------------
            features['installation_id'] = session['installation_id'].iloc[-1]
            features['session_title'] = session_title
            
            # nums in target assessment data ------------------------------------------
            if durations == []: #span of timestamp in target assessment
                features['duration_mean'] = 0
            else:
                features['duration_mean'] = np.mean(durations)
            durations.append((session.iloc[-1, 2] - session.iloc[0, 2]).seconds) 
            
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            features.update(accuracy_groups)
            accuracy_groups[str(features['accuracy_group'])] += 1
            features['accumulated_accuracy_group'] = accumulated_accuracy_group/counter if counter > 0 else 0
            accumulated_accuracy_group += features['accuracy_group']
            
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                all_assessments.append(features)
                
            counter += 1
                        
        n_of_title_eventcode = Counter(session['title_event_code']) 
        for key in n_of_title_eventcode.keys():
            title_eventcode_count[str(key)] += n_of_title_eventcode[key]
        miss += np.sum(session["misses"])

        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[session_type] += 1
            last_activitiy = session_type
    if test_set:
        return all_assessments[-1]
    return all_assessments

In [8]:
new_train = []
for i, (ins_id, user_sample) in tqdm(enumerate(train.groupby('installation_id', sort=False)), total=train.installation_id.nunique(), desc='Installation_id', position=0):
    new_train += get_data(user_sample)
new_train = pd.DataFrame(new_train)
print(new_train.shape)


(17690, 461)


In [9]:
del train

In [10]:
new_test = []
for ins_id, user_sample in tqdm(test.groupby('installation_id', sort=False), total=test.installation_id.nunique(), desc='Installation_id', position=0):
    a = get_data(user_sample, test_set=True)
    new_test.append(a)   
new_test = pd.DataFrame(new_test)
print(new_test.shape)


(1000, 461)


In [11]:
del test

# Feature selection and modelling

In [12]:
correlations = new_train.corr().abs()
correlations = correlations.mask(np.tril(np.ones(correlations.shape)).astype(np.bool))
correlations = correlations.stack().reset_index()
corr_columns = ["level_0", "level_1", "value"]
correlations.columns = corr_columns
correlations = correlations.sort_values("value", ascending=False).reset_index(drop=True)

high_corr = correlations[correlations["value"] == 1]

high_corr_features = []
for i in range(high_corr.shape[0]):
    if high_corr.iloc[i]["level_0"] not in high_corr_features and high_corr.iloc[i]["level_1"] not in high_corr_features:
        high_corr_features.append(high_corr.iloc[i]["level_0"])
    elif high_corr.iloc[i]["level_0"] in high_corr_features and high_corr.iloc[i]["level_1"] not in high_corr_features:
        high_corr_features.append(high_corr.iloc[i]["level_1"])
    elif high_corr.iloc[i]["level_0"] not in high_corr_features and high_corr.iloc[i]["level_1"] in high_corr_features:
        high_corr_features.append(high_corr.iloc[i]["level_0"])

In [13]:
X_train = new_train.drop(['accuracy_group'],axis=1) 
lbl = preprocessing.LabelEncoder()
lbl.fit(list(X_train["installation_id"]))
X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
remove_features = []
for i in X_train.columns:
    if X_train[i].std() == 0 and i not in remove_features:
        remove_features.append(i)
X_train = X_train.drop(remove_features, axis=1)
X_train = X_train[sorted(X_train.columns.tolist())]
y_train = new_train.accuracy_group
print(X_train.shape)

(17690, 435)


In [14]:
n_folds=5
skf=StratifiedKFold(n_splits = n_folds)
coefficients = []
models = []
train_qwk_scores = []
test_qwk_scores = []
xgb_params = {
    "objective" : "reg:squarederror",
    "eval_metric" : "rmse",
    "tree_learner": "serial",
    "max_depth" : 4,
    "boosting": 'gbdt',
    "num_leaves" : 13,
    "learning_rate" : 0.01,
}
#evals_result = {}
num_boost_round=100000
for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
    optR = OptimizedRounder()
    X_train2 = X_train.iloc[train_index,:]
    y_train2 = y_train.iloc[train_index]
    X_train2 = X_train2.drop(['installation_id'],axis=1)
    
    # truncation of validation data to make it closer to test data
    X_test2 = X_train.iloc[test_index,:]
    y_test2 = y_train.iloc[test_index]
    test2 = pd.concat([X_test2, y_test2], axis=1)
    test2 = test2.groupby('installation_id').apply(lambda x: x.sample(1, random_state=1223)).reset_index(drop=True)
    X_test2 = test2.drop(["accuracy_group", "installation_id"], axis=1)
    y_test2 = test2["accuracy_group"]

    xgb_train = xgb.DMatrix(X_train2, label = y_train2)
    xgb_eval = xgb.DMatrix(X_test2, label = y_test2)
    watchlist = [(xgb_train, "train"), (xgb_eval, "eval")]
    
    clf = xgb.train(
        xgb_params, xgb_train, num_boost_round, watchlist,
        early_stopping_rounds=10,
        #evals_result=evals_result,
        #feval=crps_score,
    )
    
    models.append(clf)
    train_predict = clf.predict(xgb_train, ntree_limit=clf.best_ntree_limit)
    test_predict = clf.predict(xgb_eval, ntree_limit=clf.best_ntree_limit)
    
    optR.fit(train_predict.reshape(-1,), y_train2)
    tmp_coefficients = optR.coefficients()
    print("fold_"+str(i)+" coefficients: ", tmp_coefficients)
    opt_train_preds = optR.predict(train_predict.reshape(-1, ), tmp_coefficients)
    train_qwk_score = qwk(y_train2, opt_train_preds)
    print("training qwk: ", train_qwk_score)
    opt_test_preds = optR.predict(test_predict.reshape(-1, ), tmp_coefficients)
    test_qwk_score = qwk(y_test2, opt_test_preds)
    print("validation qwk: ", test_qwk_score)
    train_qwk_scores.append(train_qwk_score)
    test_qwk_scores.append(test_qwk_score)
    coefficients.append(tmp_coefficients)
    
    if i == 0:
        feature_importance_df = pd.DataFrame(list(clf.get_score(importance_type="total_gain").keys()), columns=["Features"])
    feature_importance_df["fold_"+str(i)] = pd.DataFrame.from_dict(clf.get_score(importance_type="total_gain").values())
        
feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]

print("                             ")
print("-----------------------------")
print('coefficients: ', coefficients)
print('train qwk list:', train_qwk_scores)
print('train qwk average score:',np.mean(train_qwk_scores))
print('valid qwk list: ', test_qwk_scores)
print('valid qwk average score:',np.mean(test_qwk_scores))

[0]	train-rmse:1.85805	eval-rmse:1.81876
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 10 rounds.
[1]	train-rmse:1.84557	eval-rmse:1.80711
[2]	train-rmse:1.83326	eval-rmse:1.79561
[3]	train-rmse:1.8211	eval-rmse:1.7844
[4]	train-rmse:1.80911	eval-rmse:1.77323
[5]	train-rmse:1.79729	eval-rmse:1.76233
[6]	train-rmse:1.78561	eval-rmse:1.75146
[7]	train-rmse:1.7741	eval-rmse:1.74087
[8]	train-rmse:1.76275	eval-rmse:1.73031
[9]	train-rmse:1.75154	eval-rmse:1.72003
[10]	train-rmse:1.74049	eval-rmse:1.70967
[11]	train-rmse:1.72967	eval-rmse:1.69955
[12]	train-rmse:1.7189	eval-rmse:1.68952
[13]	train-rmse:1.70838	eval-rmse:1.67968
[14]	train-rmse:1.698	eval-rmse:1.66999
[15]	train-rmse:1.68767	eval-rmse:1.66041
[16]	train-rmse:1.67758	eval-rmse:1.65099
[17]	train-rmse:1.66763	eval-rmse:1.64173
[18]	train-rmse:1.65773	eval-rmse:1.63254
[19]	train-rmse:1.64807	eval-rmse:1.62354
[20]	train-rmse:1.63843	eval-rmse

In [15]:
#my_model = xgb.XGBRegressor(**xgb_params).fit(X_train2, y_train2)
#perm = PermutationImportance(my_model, random_state=1).fit(X_test2, y_test2)
#eli5.show_weights(perm, feature_names = X_test2.columns.tolist(), top = 500)

In [16]:
#feature_importance_df.sort_values("Average", ascending=False).reset_index(drop=True).head(500)

In [17]:
print(X_train.shape)
print("")
print(", ".join('%s'%x for x in X_train.columns))

(17690, 435)

0, 1, 12 Monkeys_2000, 2, 3, Activity, Air Show, Air Show_2000, Air Show_2020, Air Show_2030, Air Show_2060, Air Show_2070, Air Show_2075, Air Show_3010, Air Show_3020, Air Show_3021, Air Show_3110, Air Show_3120, Air Show_3121, Air Show_4010, Air Show_4020, Air Show_4070, Air Show_4090, Air Show_4100, Air Show_4110, All Star Sorting, All Star Sorting_2000, All Star Sorting_2020, All Star Sorting_2025, All Star Sorting_2030, All Star Sorting_3010, All Star Sorting_3020, All Star Sorting_3021, All Star Sorting_3110, All Star Sorting_3120, All Star Sorting_3121, All Star Sorting_4010, All Star Sorting_4020, All Star Sorting_4030, All Star Sorting_4035, All Star Sorting_4070, All Star Sorting_4080, All Star Sorting_4090, All Star Sorting_4095, Assessment, Balancing Act_2000, Bird Measurer (Assessment)_2000, Bird Measurer (Assessment)_2010, Bird Measurer (Assessment)_2020, Bird Measurer (Assessment)_2030, Bird Measurer (Assessment)_3010, Bird Measurer (Assessment)_3020, Bird 

# prediction

In [18]:
X_test = new_test.drop(["installation_id", "accuracy_group"], axis=1)
X_test = X_test.drop(remove_features, axis=1)
X_test = X_test[sorted(X_test.columns.tolist())]
pred_value = np.zeros([X_test.shape[0]])
test_coefficients = np.mean(coefficients, axis=0)
X_test = xgb.DMatrix(X_test)
for model in models:
    pred_value += model.predict(X_test, ntree_limit= model.best_ntree_limit) /n_folds
test_pred_class= optR.predict(pred_value.reshape(-1, ), test_coefficients)
sample_submission["accuracy_group"] = test_pred_class
sample_submission.to_csv('submission.csv', index=False)

In [19]:
sample_submission["accuracy_group"].value_counts()

3    451
2    290
1    171
0     88
Name: accuracy_group, dtype: int64